In [1]:
! pip install huggingface_hub
! pip install datasets
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 4.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 12.9 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 49.

# get data

In [3]:
! pip install -q kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"apollopubg","key":"3c6e2519fa3fc29b6143e88f14a90408"}'}

In [4]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d thedevastator/dailydialog-unlock-the-conversation-potential-in

  0% 0.00/2.46M [00:00<?, ?B/s]
100% 2.46M/2.46M [00:00<00:00, 83.3MB/s]


In [5]:
! unzip dailydialog-unlock-the-conversation-potential-in.zip

Archive:  dailydialog-unlock-the-conversation-potential-in.zip
  inflating: test.csv                
  inflating: train.csv               
  inflating: validation.csv          


In [7]:
import pandas as pd

In [16]:
df = pd.read_csv("test.csv")
df.iloc[959,0]

"['Excuse me , what do you want for this bag ? '\n ' Let me see . The one goes for $ 17 . ' ' Would you take $ 12 ? '\n ' No , I can ’ t let you have it for 12.How about 14 ? ' ' 13.50 ? '\n ' Sorry , $ 14 is the lowest I can go . ']"

,dialog,act,emotion
0,"['Hey man , you wanna buy some weed ? ' ' Some...",[3 2 3 4 3 4 3 2 3 4 2 3],[0 6 0 0 0 0 0 0 0 0 3 0]
1,['The taxi drivers are on strike again . ' ' W...,[1 2 1 1],[0 0 0 0]
2,"[""We've managed to reduce our energy consumpti...",[1 2 1 2 1 2 1],[0 0 0 0 0 0 0]
3,"['Believe it or not , tea is the most popular ...",[1 1 1 1 2 2 2 2 1 1 1 3 4 3],[0 0 0 0 0 0 0 0 0 4 0 0 4 4]
4,['What are your personal weaknesses ? '\n ' I ...,[2 1 2 1 2 1 2 1],[0 0 0 0 0 0 0 4]


In [17]:
import re

In [28]:
def make_dilogue(data):
  dialogue =[]
  for i in data["dialog"]:
    arr = []
    num =1
    data = re.split("(' '|'\\n ')", i[2:-2])[::2]
    for j in data:
      dictt = {
          "speaker":num,
          "text":j
      }
      if num == 1 :
        num =2
      else :
        num=1
      arr.append(dictt)
    dialogue.append(arr)
  return dialogue

In [29]:
dialogue = make_dilogue(df)

In [30]:
dialogue[:2]

[[{'speaker': 1, 'text': 'Hey man , you wanna buy some weed ? '},
  {'speaker': 2, 'text': ' Some what ? '},
  {'speaker': 1,
   'text': ' Weed ! You know ? Pot , Ganja , Mary Jane some chronic ! '},
  {'speaker': 2, 'text': ' Oh , umm , no thanks . '},
  {'speaker': 1,
   'text': ' I also have blow if you prefer to do a few lines . '},
  {'speaker': 2, 'text': ' No , I am ok , really . '},
  {'speaker': 1,
   'text': ' Come on man ! I even got dope and acid ! Try some ! '},
  {'speaker': 2,
   'text': ' Do you really have all of these drugs ? Where do you get them from ? '},
  {'speaker': 1,
   'text': ' I got my connections ! Just tell me what you want and I ’ ll even give you one ounce for free . '},
  {'speaker': 2, 'text': ' Sounds good ! Let ’ s see , I want . '},
  {'speaker': 1, 'text': ' Yeah ? '},
  {'speaker': 2,
   'text': ' I want you to put your hands behind your head ! You are under arrest ! '}],
 [{'speaker': 1, 'text': 'The taxi drivers are on strike again . '},
  {'sp

In [32]:
def chatbotdata_process(data):
  from transformers import Conversation
  question=[]
  answer = []

  for i in data:
    num =1
    for index,j in enumerate(i[:-1]):
      if index % 2 == 0:
        past_qus=[]
        past_answ = []
        if index > 0 :
          for index2, k in enumerate(i[:index]):
            if index2 % 2 ==0:
              past_qus.append(k["text"][:10])
            else :
              past_answ.append(k["text"][:10])
        conversation = Conversation(j["text"][:10],generated_responses=past_answ,past_user_inputs=past_qus,conversation_id=num)
        num = num +1
        question.append("<|beginningoftext|>"+"|".join(re.split("\\n",str(conversation))[1:])[:-1]+"<|endoftext|>")
        answer.append("|bot >>"+i[index+1]["text"])
  return [question,answer]

In [33]:
[question,answer] = chatbotdata_process(dialogue)

In [34]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

from torch.optim import Adam
from torch.utils.data import DataLoader
import tqdm
import torch
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
SPECIAL_TOKENS = {
        'bos_token': '<|beginningoftext|>',
        'eos_token': '<|endoftext|>',
        'sep_token': '<|sep|>',
        'pad_token': '<|pad|>',
        'unk_token': '<|unk|>'
        }
tokenizer.add_special_tokens(SPECIAL_TOKENS)


speaker_token = '|user >>'
bot_token = '|bot >>'
tokenizer.add_tokens([speaker_token, bot_token], special_tokens=True)


model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))

model = model.to(device)

In [35]:
model.resize_token_embeddings(len(tokenizer))

Embedding(50262, 768)

In [42]:
def process(num):
  from datasets import Dataset
  toke = tokenizer(text=question[:num],text_target=answer[:num],truncation=True,max_length=50,padding=True)
  dataset = Dataset.from_dict(toke)
  return dataset
train_data = process(100)
test_data =  process(20)

In [43]:
train_data

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 100
})

In [44]:
from transformers import Seq2SeqTrainingArguments,Seq2SeqTrainer,DataCollatorForSeq2Seq
collator = DataCollatorForSeq2Seq(tokenizer, model = model)

In [45]:
args = Seq2SeqTrainingArguments(
    "conversationv8",
    per_device_train_batch_size=28,
    per_device_eval_batch_size=28,
    optim="adamw_torch",
    learning_rate=2e-3,
    # weight_decay=0.01,
    # evaluation_strategy="epoch",
    # logging_strategy="epoch",
    # log_level="error",
    # predict_with_generate=True,
    num_train_epochs=1,
    # push_to_hub = True,
  )

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [46]:
train = Seq2SeqTrainer(
    model,
    args,
    train_dataset = train_data,
    eval_dataset = test_data,
    data_collator = collator,
    tokenizer = tokenizer,
    # compute_metrics = compute_metrics
  )

In [47]:
train.train()

***** Running training *****
  Num examples = 100
  Num Epochs = 1
  Instantaneous batch size per device = 28
  Total train batch size (w. parallel, distributed & accumulation) = 28
  Gradient Accumulation steps = 1
  Total optimization steps = 4
  Number of trainable parameters = 124443648


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=4, training_loss=40.44907760620117, metrics={'train_runtime': 269.7569, 'train_samples_per_second': 0.371, 'train_steps_per_second': 0.015, 'total_flos': 2551680000000.0, 'train_loss': 40.44907760620117, 'epoch': 1.0})

In [48]:
tokenizer("|user >> hi ")

{'input_ids': [50260, 5303, 220], 'attention_mask': [1, 1, 1]}